In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ResInception_cifar10 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [3]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.1
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 2.78987
step 1, cross_entropy 25.7324
step 2, cross_entropy 2.26216
step 3, cross_entropy 4.65602
step 4, cross_entropy 2.89713
step 5, cross_entropy 2.20805
step 6, cross_entropy 2.17606
step 7, cross_entropy 2.69592
step 8, cross_entropy 2.44378
step 9, cross_entropy 2.46399
step 10, cross_entropy 2.30059
step 11, cross_entropy 2.2801
step 12, cross_entropy 2.33627
step 13, cross_entropy 2.32098
step 14, cross_entropy 2.33871
step 15, cross_entropy 2.34131
step 16, cross_entropy 2.35168
step 17, cross_entropy 2.33259
step 18, cross_entropy 2.28559
step 19, cross_entropy 2.28463


In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.11112
test accuracy 0.122
step 399, cross_entropy 2.21768
test accuracy 0.177
step 599, cross_entropy 2.13828
test accuracy 0.19
step 799, cross_entropy 1.75827
test accuracy 0.275
step 999, cross_entropy 1.9416
test accuracy 0.294
step 1199, cross_entropy 1.51351
test accuracy 0.289
step 1399, cross_entropy 1.95171
test accuracy 0.315
step 1599, cross_entropy 2.07078
test accuracy 0.385
step 1799, cross_entropy 1.5637
test accuracy 0.356
step 1999, cross_entropy 1.87015
test accuracy 0.375
step 2199, cross_entropy 2.07582
test accuracy 0.4
step 2399, cross_entropy 1.61826
test accuracy 0.487
step 2599, cross_entropy 1.41523
test accuracy 0.462
step 2799, cross_entropy 1.69277
test accuracy 0.356
step 2999, cross_entropy 1.1868
test accuracy 0.473
step 3199, cross_entropy 1.55049
test accuracy 0.511
step 3399, cross_entropy 1.68035
test accuracy 0.476
step 3599, cross_entropy 1.48903
test accuracy 0.461
step 3799, cross_entropy 2.08369
test accuracy 0.443
step

In [6]:
rate = 0.002
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.384449
test accuracy 0.787
step 399, cross_entropy 0.316539
test accuracy 0.793
step 599, cross_entropy 0.440458
test accuracy 0.796
step 799, cross_entropy 0.603934
test accuracy 0.778
step 999, cross_entropy 0.459948
test accuracy 0.791
step 1199, cross_entropy 0.782961
test accuracy 0.786
step 1399, cross_entropy 0.47534
test accuracy 0.792
step 1599, cross_entropy 0.34572
test accuracy 0.785
step 1799, cross_entropy 0.747967
test accuracy 0.803
step 1999, cross_entropy 0.293292
test accuracy 0.812
step 2199, cross_entropy 0.702221
test accuracy 0.778
step 2399, cross_entropy 0.15031
test accuracy 0.787
step 2599, cross_entropy 0.368393
test accuracy 0.805
step 2799, cross_entropy 0.497139
test accuracy 0.792
step 2999, cross_entropy 0.591614
test accuracy 0.81
step 3199, cross_entropy 0.286178
test accuracy 0.791
step 3399, cross_entropy 1.18958
test accuracy 0.778
step 3599, cross_entropy 0.629749
test accuracy 0.799
step 3799, cross_entropy 0.193661
test

In [7]:
rate = 0.0004
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.30501
test accuracy 0.812
step 399, cross_entropy 0.123881
test accuracy 0.807
step 599, cross_entropy 0.10988
test accuracy 0.807
step 799, cross_entropy 0.436863
test accuracy 0.795
step 999, cross_entropy 0.492564
test accuracy 0.782
step 1199, cross_entropy 0.270226
test accuracy 0.811
step 1399, cross_entropy 0.45728
test accuracy 0.796
step 1599, cross_entropy 0.531902
test accuracy 0.824
step 1799, cross_entropy 0.438358
test accuracy 0.811
step 1999, cross_entropy 0.152156
test accuracy 0.807
step 2199, cross_entropy 0.352801
test accuracy 0.806
step 2399, cross_entropy 0.498936
test accuracy 0.806
step 2599, cross_entropy 0.654135
test accuracy 0.795
step 2799, cross_entropy 0.240304
test accuracy 0.81
step 2999, cross_entropy 0.452577
test accuracy 0.827
step 3199, cross_entropy 0.119517
test accuracy 0.799
step 3399, cross_entropy 0.0844067
test accuracy 0.809
step 3599, cross_entropy 0.264714
test accuracy 0.83
step 3799, cross_entropy 0.260095
tes